In [ ]:
%matplotlib inline

In [ ]:
import json
import os
from copy import copy
from functools import partial
from os.path import basename, join
from subprocess import call
from urllib.request import urlopen

import pyproj
import requests

import geopandas as gpd
import pandas as pd
import us
from shapely.geometry import LineString, Point, shape
from shapely.ops import transform

HCRIS facilities are geocoded using the [Google Maps](https://developers.google.com/maps/documentation/geocoding/start) and [Mapbox](https://docs.mapbox.com/api/search/) geocoding APIs. Google Maps is the primary source while Mapbox is used as a fallback option. We validate the geocoding results by checking their spatial overlap with administrative regions (states, counties, zip codes) that they are expected to fall into. We use three different level because of underlying inaccuracies in the HCRIS data.

Geocoding steps:
- Construct a query based on address info in the HCRIS record
- Find the geometries that the output point is expected to fall into based on the HCRIS record(state, county, zip code)
- Geocode using the Google Maps API
- If the point falls into the expected County or Zip Code, proceed, otherwise check it against its expected state geometry.
- Track the source of confirmation for each facility so that we can manually inspect the results that were confirmed at the higher geographic level (i.e. state)

### Input Datasets

*states geodataframe*

In [ ]:
states = gpd.read_file('https://gist.githubusercontent.com/simonkassel/d091fc86253b65c68bb644443c74f366/raw/001b6ad8232e2ecfdc5fbd46a5ef8f2a9642e94d/us_states.geojson')
states.rename(columns={'NAME': 'ST_NAME', 'abbr': 'ST_ABBR', 'STATEFP':'STATE_FP'}, inplace=True)
states['ST_NAME'] = states['ST_NAME'].apply(lambda x: x.upper())

*counties geodataframe*

In [ ]:
def gdf_from_geojson(geojson, crs='epsg:4326'):
    """Convert a GeoJSON dict to GeoDataFrame"""
    def _f(f):
        f['properties']['geometry'] = shape(f['geometry'])
        return f['properties']
    
    return gpd.GeoDataFrame([_f(f) for f in geojson['features']], crs=crs)

fips_to_abbr = {st.fips: st.abbr for st in us.states.STATES_AND_TERRITORIES if st.fips}

In [ ]:
with urlopen("https://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_050_00_20m.json") as url:
    data = json.loads(url.read().decode("ISO-8859-1"))
counties = gdf_from_geojson(data)
counties.rename(columns={'STATE':'STATE_FP', 'COUNTY': 'COUNTY_FP', 'NAME': 'COUNTY_NAME'}, inplace=True)
counties['COUNTY_NAME'] = counties['COUNTY_NAME'].apply(lambda x: x.upper())
counties['ST_ABBR'] = counties['STATE_FP'].apply(lambda x: fips_to_abbr[x])

*zip code convex hull geodataframe*

The zip code geojson file for the whole US was prohibitively large so we reduced the size by simplifying the polygons into their [convex hulls](https://en.wikipedia.org/wiki/Convex_hull). This dramatically reduced the file size while keeping enough spatial information for the simple task of validating basic location.

In [ ]:
zip_convex_hulls = gpd.read_file('../data/us_zip_codes-convex_hulls.geojson')

*hcris dataframe*

In [ ]:
if not os.path.isdir('download_data'):
    os.mkdir('download_data')

url = 'http://downloads.cms.gov/files/hcris/hosp10-reports.zip'
z = basename(url)
c = 'download_data/HOSPITAL10_PROVIDER_ID_INFO.CSV'

if not os.path.exists(c):
    call('wget -O download_data/{} {}'.format(z, url), shell=True)
    call('cd download_data && unzip -o {}'.format(z), shell=True)
    
hcris = pd.read_csv(c)

# use 5 digit zip codes to match the us zip code geojson
hcris['Zip_Code'] = hcris['Zip_Code'].apply(lambda x: x.split('-')[0])
hcris.rename(columns = {'City': 'CITY_NAME', 
                        'State': 'ST_ABBR', 
                        'County': 'COUNTY_NAME', 
                        'Zip_Code': 'ZIP_CODE'}, inplace=True)

In [ ]:
# provider num should be 6 char so need to zfill
hcris['PROVIDER_NUMBER'] = hcris['PROVIDER_NUMBER'].apply(lambda x: str(x).zfill(6))

# Rename this column to match up with reports
hcris = hcris.rename(columns={'PROVIDER_NUMBER': 'Provider Number'})

### geocoding

In [ ]:
# Either replace the empty strings in the subsequent two lines with the appropriate API keys or set them as 
# environment variables
google_key = ''
mapbox_key = ''

google_key = os.getenv('GOOGLE_API_KEY', google_key)
mapbox_key = os.getenv('MAPBOX_API_KEY', mapbox_key)

In [ ]:
def google_geocode_str(name, addr, city, state, county, zip_code, key, condense=False):
    """
    Generate a Google Maps query url from HCRIS fields and api key
    
    `condense` is an optional boolean parameter that enables construction of the URL with 
    all of the components combined into the address field rather than as separate components
    this is an option if the latter fails
    """
    
    country = 'US'
    if state in ('PR', 'GU'):
        country = state

    base = 'https://maps.googleapis.com/maps/api/geocode/json?'
    if condense:
        address_str = f'address={name}, {addr}, {city}, {state}, {county} county, {zip_code}'
        components_str = '&components=country:{}'.format(country)
    else:
        address_str = 'address={}, {}'.format(name, addr)
        components_str = f'&components=country:{country}|locality:{city}|administrative_area:{state}|administrative_area:{county} county|postal_code:{zip_code}'
    
    key_str = '&key={}'.format(key)
    
    return base + address_str + components_str + key_str

In [ ]:
def mapbox_geocode_str(name, addr, city, state, county, zip_code, key):
    """
    Generate a Mapbox query url from HCRIS fields and api key
    """
    country = 'US'
    if state in ('PR', 'GU'):
        country = state
        
    base = 'https://api.mapbox.com/geocoding/v5/mapbox.places/'
    address_str = f'{name},{addr},{city},{state},{county} county,{zip_code}.json?'
    components_str = 'country={}&limit=5'.format(country)
    key_str = '&access_token={}'.format(key)
    
    query_str = base + address_str + components_str + key_str
    # mapbox urls are limited to 256 characters or fewer. Strategically remove component
    # of the address if necessary to get under the threshold
    if len(query_str) > 256:
        query_str = query_str.replace('{},'.format(name), '')
        if len(query_str) > 256:
            query_str = query_str.replace('{} county,'.format(county), '')
    
    return query_str

In [ ]:
# Change this test flag to `True` in order to run a smaller subset of all HCRIS facilities
# to make sure everythin is worlking correctly
test = False
if test:
    hcris = hcris.copy()
    hcris = hcris.head(100)

iterate over all facilities storing the geocode results and background information

In [ ]:
geometry = []
source = []
confs = []

for k, r in hcris.iterrows():
    
    county_geom = None
    zip_geom = None
    state_geom = None
    
    if isinstance(r['COUNTY_NAME'], str):
        county = counties[(counties['ST_ABBR'] == r['ST_ABBR']) & (counties['COUNTY_NAME'] == r['COUNTY_NAME'])]
        if len(county) == 1:
            county_geom = county['geometry'].values[0]
        
        elif len(county) < 1:
            # some counties are encoded slightly differently in hcris dataset
            rcn = r['COUNTY_NAME'].replace(' COUNTY', '')
            county = counties[(counties['ST_ABBR'] == r['ST_ABBR']) & (counties['COUNTY_NAME'] == rcn)]
            if len(county) == 1:
                county_geom = county['geometry'].values[0]
        
        elif len(county) > 1:
            county = county[county['LSAD'] == 'County']
            if len(county) == 1:
                county_geom = county['geometry'].values[0]
    
    # using zip code is necessary because many facilities do not fall within counties or have
    # incorrect county labels in the hcris dataset
    if isinstance(r['ZIP_CODE'], str):
        zips = zip_convex_hulls[zip_convex_hulls['ZIP_CODE'] == r['ZIP_CODE']]
        if len(zips) == 1:
            zip_geom = zips['geometry'].values[0]

    state_geom = states[states['ST_ABBR'] == r['ST_ABBR']]['geometry'].values[0]
    
    
    # workaround for one exception in PR
    if r['Street_Addr'] == 'ROAD 172 EXIT TO CIDRA':
        r['Street_Addr'] = ''
        
    args = [r['HOSP10_Name'], r['Street_Addr'], r['CITY_NAME'], r['ST_ABBR'], r['COUNTY_NAME'], r['ZIP_CODE']]
    args = [str(a).replace('#', '').replace('/', ' ').replace(';', '').replace('?', '') for a in args]
    
    def _p(search_str):
        """
        Convert http request str into a validated shapely point
        """
        response = requests.get(search_str)
        if response.status_code == 422:
            print('Mapbox query string too long, relying on google maps')
            
        if response.status_code != 200:
            print(response.json())
            raise Exception('Status code exeption: {}'.format(response.status_code))
        
        
        response = response.json()
        if 'results' in response:
            for result in response['results']:
                y, x = result['geometry']['location'].values()
                hosp_point = Point(x, y)
                if county_geom:
                    if hosp_point.within(county_geom):
                        return (hosp_point, 'county')
                if zip_geom:
                    if hosp_point.within(zip_geom):
                        return (hosp_point, 'zip code')
                if state_geom:
                    if hosp_point.within(state_geom):
                        return (hosp_point, 'state')
        else:
            for feature in response['features']:
                hosp_point = Point(feature['center'])
                hosp_point = Point(x, y)
                if county_geom:
                    if hosp_point.within(county_geom):
                        return (hosp_point, 'county')
                if zip_geom:
                    if hosp_point.within(zip_geom):
                        return (hosp_point, 'zip code')
                if state_geom:
                    if hosp_point.within(state_geom):
                        return (hosp_point, 'state')
        
        return (None, None)
    
    # Google
    # Try google first
    google_point, confirmation = _p(google_geocode_str(*args + [google_key]))
    if not google_point:
        # and try both different types of google request construction
        google_point, confirmation = _p(google_geocode_str(*args + [google_key, True]))
    
    # Mapbox
    # only try mapbox if google didn't return a valid point
    mapbox_point = None
    if not google_point:
        mapbox_point, confirmation = _p(mapbox_geocode_str(*args + [mapbox_key]))
        
    if google_point:
        source.append('google')
        geometry.append(google_point)
    elif mapbox_point:
        source.append('mapbox')
        geometry.append(mapbox_point)
    else:
        print('No coordinates found for row:')
        print(r)
        print()
        source.append(None)
        geometry.append(None)
        
    confs.append(confirmation)
    
    if (k + 1) % 100 == 0:
        print('Geocoded [{}] of {} hospitals.'.format(k + 1, len(hcris)))

# add geocoding info to hcris dataset and convert into GeoDataFrame
hcris['source'] = source
hcris['geometry'] = geometry
hcris['confirmation_source'] = confs
gdf = gpd.GeoDataFrame(hcris, crs='epsg:4326')

f = '../data/usa_facilities_hcris_geocoded_v2.geojson'
if test:
    f = f.replace('.geojson', '-TEST.gejson')

# save off file as GeoJSON
gdf.to_file(f, driver='GeoJSON')